In [4]:
import sys
sys.path.append('../')

import pandas as pd
import backtrader as bt
from datetime import datetime
import matplotlib.pyplot as plt
from Tools.backtest_tools import *
import plotly.graph_objects as go
import pandas_ta as ta 
from Tools.plotting import CandlePlot

In [5]:
# Load the data and convert the timestamp to datetime
df = pd.read_csv('../DB/5m/INJUSDT_5m.csv', parse_dates=True) 

# df['datetime'] = pd.to_datetime(df['datetime'], unit='ms')
# # # Convert 'date' to datetime format
df['datetime'] = pd.to_datetime(df['datetime'])  # Assuming the 'date' is in milliseconds

# # # Set 'date' as index
df.set_index('datetime', inplace=True)

# df.iloc[0]
# df.dtypes
# df.head()
df = df[:].copy()

In [6]:
    
# -- Populate indicators --
df['EMA_slow'] = ta.ema(df.close, length=50)
df['EMA_fast'] = ta.ema(df.close, length=30)
df['rsi'] = ta.rsi(df.close, length=10)
df['ATR']= ta.atr(df.high, df.low, df.close, length=7)
m_bbands = ta.bbands(df.close, length=15, std=1.5) 

df = df.join(m_bbands)        

In [7]:
df

,open,high,low,close,volume,quote_volume,EMA_slow,EMA_fast,rsi,ATR,BBL_15_1.5,BBM_15_1.5,BBU_15_1.5,BBB_15_1.5,BBP_15_1.5
datetime,,,,,,,,,,,,,,,
2022-11-21 03:15:00,1.564,1.564,1.559,1.559,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-11-21 03:20:00,1.559,1.560,1.556,1.557,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-11-21 03:25:00,1.557,1.564,1.553,1.563,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-11-21 03:30:00,1.563,1.576,1.563,1.576,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-11-21 03:35:00,1.576,1.577,1.575,1.577,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-09-04 04:05:00,15.645,15.651,15.603,15.608,0,0,15.665729,15.608649,50.878176,0.050655,15.558226,15.610200,15.662174,0.665893,0.478835
2024-09-04 04:10:00,15.608,15.630,15.600,15.627,0,0,15.664211,15.609833,53.960180,0.047704,15.578470,15.617133,15.655797,0.495141,0.627597
2024-09-04 04:15:00,15.627,15.631,15.572,15.576,0,0,15.660751,15.607650,45.454491,0.049318,15.577603,15.616867,15.656131,0.502841,-0.020409


In [ ]:
def ema_signal(df, current_candle, back_candles):
    df_slice = df.reset_index().copy()
    start = max(0, current_candle - back_candles)
    end = current_candle
    relevant_rows = df_slice.iloc[start:end]

    if all(relevant_rows['EMA_fast'] < relevant_rows['EMA_slow']):
        return 1
    if all(relevant_rows['EMA_fast'] > relevant_rows['EMA_slow']):
        return 2
    return 0

df = df[-30000:-1]
df.reset_index(inplace=True, drop=True)
df['EMA_signal'] = 0
df.loc[20:, 'EMA_signal'] = df.loc[20:].apply(lambda row: ema_signal(df, row.name, 7), axis=1)

C:\Users\idf-v\AppData\Local\Temp\ipykernel_52904\2826484621.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['EMA_signal'] = 0


In [ ]:
df

In [ ]:
def total_signal(df, current_candle, backcandles):
    if (ema_signal(df, current_candle, backcandles)==2
        and df.close[current_candle]<=df['BBL_15_1.5'][current_candle]
        #and df.RSI[current_candle]<60
        ):
            return 2
    if (ema_signal(df, current_candle, backcandles)==1
        and df.close[current_candle]>=df['BBU_15_1.5'][current_candle]
        #and df.RSI[current_candle]>40
        ):
    
            return 1
    return 0
        
df['TotalSignal'] = df.apply(lambda row: total_signal(df, row.name, 7), axis=1)


In [ ]:
def pointpos(x):
    if x['TotalSignal']==2:
        return x['low']-1e-3
    elif x['TotalSignal']==1:
        return x['high']+1e-3
    else:
        return np.nan

df['pointpos'] = df.apply(lambda row: pointpos(row), axis=1)

In [11]:
dfplt = df[450:700]
cp = CandlePlot(dfplt)
cp.add_points()
cp.add_traces(['EMA_slow', 'EMA_fast', 'BBL_15_1.5', 'BBU_15_1.5', 'BBM_15_1.5'])


In [ ]:
cp.show_plot()